# Part 6.1 - Prepare Dataframe
---

### Papers Past Topic Modeling
<br/>


Ben Faulks - bmf43@uclive.ac.nz

Xiandong Cai - xca24@uclive.ac.nz

Yujie Cui - ycu23@uclive.ac.nz

In [1]:
import sys
sys.path.insert(0, '../utils')
import pandas as pd
pd.set_option('display.max_columns', 120)
pd.set_option('display.max_colwidth', 120)
from pyspark.sql import functions as F
from pyspark.sql.types import *
from utils import conf_pyspark, load_dataset
from utils_preplot import preplot, load_doctopic

# intiate PySpark
sc, spark = conf_pyspark()

sc

[('spark.driver.host', 'x99.hub'),
 ('spark.app.name', 'local'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.driver.memory', '62g'),
 ('spark.master', 'local[*]'),
 ('spark.driver.port', '35056'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.id', 'local-1548394617030'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.cores', '6'),
 ('spark.driver.maxResultSize', '4g')]


<SparkContext master=local[*] appName=local>

**This part we generate the dataframe that part 6.2 will be used:**
* dominant topic per year dataframe;
* average topic weight per year dataframe;
* dominant topic per month dataframe (only for WWI analysis);
* average topic weight per month dataframe (only for WWI analysis);

**After training topic models, we got the topic words list (**`topicKeys.txt`**) and the doc-topic matrix (**`docTopics.txt`**). The doc-topic matrix is a docmument based file which means we could connect the topic weights in the matrix and the metadata (date, region, publisher) in the meta dataset, then the combined dataset could apply to accomplish many data mining or statistical works, for instance, the granularity of time can achieve 1 day, which could perform time series analsys. Now we load those data to generate dataframe for analysys and visualization.**

## 1 Prepare Dataframe for Train Set

### 1.1 Load data

**Load metadata ("id", "region" and "date"):**

In [2]:
df_meta = load_dataset('meta', spark).select(F.col('id').alias('id_'), F.col('region'), F.col('date'))

In [3]:
df_meta.limit(5).toPandas().head()

,id_,region,date
0,1854215,Auckland,1862-06-14
1,1854224,Auckland,1862-07-03
2,1854233,Auckland,1863-08-01
3,1854245,Auckland,1863-09-01
4,1854248,Auckland,1863-09-01


**Load topic words list:**

In [39]:
path = r'../models/train/topicKeys.txt'

data_schema = StructType([
    StructField('topic', IntegerType()),
    StructField('weight_', FloatType()),
    StructField('words', StringType())
])

df_topics = (
    spark.read.format("com.databricks.spark.csv")
    .option("header", "false")
    .option("inferSchema", "false")
    .option("delimiter", "\t")
    .schema(data_schema)
    .load(path)
)
topic_number = df_topics.count()

In [5]:
print('Shape of dataframe: ({}, {})'.format(topic_number, len(df_topics.columns)))
df_topics.limit(10).toPandas().head(10)

Shape of dataframe: (200, 3)


,topic,weight_,words
0,0,0.00755,council city borough mayor town water street councillor works supply
1,1,0.00467,blenheim marlborough picton street wairau griffiths market road high havelock
2,2,0.01173,years man time london made ago great lady day young
3,3,0.00726,wool sheep stock company agents sale produce dip sales grain
4,4,0.00409,freight apply passage steam company passengers new_zealand tons agents wellington
5,5,0.00478,white black blue silk dress made colour coat green wear
6,6,0.00489,church rev bishop work mission synod presbyterian meeting congregation churches
7,7,0.00678,nelson motueka richmond takaka waimea trafalgar collingwood wakefield bridge-street hardy
8,8,0.00269,tons cases case schooner bags agent agents sacks bales passengers
9,9,0.01238,men soldiers war service military army new_zealand camp work fund


**Load doc-topic matrix:**

In [6]:
path = r'../models/train/docTopics.txt'

# generate new column names
columns = [str(x) for x in list(range(topic_number))]
columns.insert(0, 'id')
columns.insert(0, 'index')

# load data
df_doctopic = (
    spark.read.format("com.databricks.spark.csv")
    .option("header", "false")
    .option("inferSchema", "true")
    .option("delimiter", "\t")
    .load(path)
)

# change columns name and drop # column which is table index and useless
df_doctopic = df_doctopic.toDF(*columns)

In [7]:
print('Shape of dataframe: ({}, {})'.format(df_doctopic.count(), len(df_doctopic.columns)))

df_doctopic.limit(5).toPandas().head()

Shape of dataframe: (1511227, 202)


,index,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,...,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199
0,0,1854215,0.000129,0.000080,0.000201,0.000124,0.000070,0.000082,0.000084,0.000116,0.000046,0.000212,0.000119,0.273705,0.000197,0.000099,0.000180,0.000118,0.000020,0.000040,0.000093,0.000061,0.000061,0.000103,0.000176,0.034306,0.017247,0.000140,0.000142,0.000057,0.000063,0.000032,0.000038,0.000143,0.000098,0.000150,0.000119,0.000114,0.000218,0.000117,0.000052,0.000248,0.000107,0.000144,0.000146,0.000124,0.000083,0.000064,0.000316,0.000131,0.000053,0.000119,0.000190,0.000033,0.000087,0.000063,0.000112,0.000112,0.000121,0.000192,...,0.000109,0.000092,0.000244,0.000159,0.000111,0.000169,0.034433,0.000119,0.000106,0.000172,0.000197,0.000143,0.000146,0.000274,0.000075,0.000111,0.000126,0.000113,0.000157,0.000088,0.000109,0.000189,0.000122,0.000151,0.000077,0.119790,0.000096,0.000078,0.000082,0.000111,0.000047,0.000218,0.017301,0.000050,0.000173,0.000080,0.000284,0.000188,0.000226,0.000104,0.000085,0.000109,0.000107,0.000083,0.000148,0.000043,0.000112,0.000068,0.000041,0.000122,0.000102,0.000160,0.000111,0.000090,0.000071,0.000059,0.000079,0.000170,0.000103,0.000125
1,1,1854224,0.000170,0.000105,0.000264,0.000163,0.000092,0.000107,0.000110,0.000152,0.000060,0.000278,0.000157,0.000301,0.000259,0.000130,0.000237,0.000155,0.000027,0.000052,0.000122,0.000080,0.000081,0.000135,0.000231,0.000160,0.000204,0.000183,0.000187,0.000076,0.000083,0.000042,0.000049,0.000188,0.000129,0.000197,0.000156,0.966307,0.000286,0.000154,0.000069,0.000326,0.000140,0.000189,0.000192,0.000163,0.000109,0.000084,0.000415,0.000172,0.000070,0.000156,0.000250,0.000043,0.000114,0.000083,0.000148,0.000147,0.000159,0.000252,...,0.000143,0.000121,0.000321,0.000209,0.000146,0.000222,0.000327,0.000157,0.000139,0.000226,0.000258,0.000188,0.000191,0.000360,0.000098,0.000145,0.000165,0.000148,0.000206,0.000115,0.000143,0.000248,0.000160,0.000198,0.000101,0.000190,0.000126,0.000103,0.000107,0.000145,0.000061,0.000286,0.000274,0.000066,0.000227,0.000105,0.000374,0.000247,0.000297,0.000137,0.000112,0.000144,0.000141,0.000109,0.000194,0.000057,0.000148,0.000089,0.000054,0.000160,0.000134,0.000211,0.000146,0.000119,0.000094,0.000078,0.000103,0.000223,0.000136,0.000164
2,2,1854233,0.000052,0.000032,0.000080,0.000050,0.000028,0.000033,0.000033,0.000046,0.000018,0.000084,0.000048,0.000092,0.020555,0.000040,0.000072,0.000047,0.000008,0.000016,0.102422,0.000024,0.000025,0.000041,0.000070,0.000049,0.000062,0.000056,0.000057,0.000023,0.000025,0.000013,0.000015,0.000057,0.000039,0.000060,0.000048,0.000045,0.000087,0.000047,0.000021,0.000099,0.000043,0.000057,0.000058,0.000049,0.000033,0.000025,0.000126,0.000052,0.000021,0.000048,0.000076,0.000013,0.000035,0.000025,0.000045,0.000045,0.000048,0.000076,...,0.000043,0.000037,0.000098,0.000064,0.000044,0.000068,0.382336,0.000048,0.000042,0.000069,0.000079,0.000057,0.000058,0.000109,0.000030,0.000044,0.000050,0.000045,0.000063,0.000035,0.000044,0.000075,0.000049,0.000060,0.000031,0.000058,0.000038,0.000031,0.000033,0.000044,0.000019,0.000087,0.000083,0.000020,0.000069,0.000032,0.000114,0.047855,0.000090,0.000042,0.000034,0.000044,0.000043,0.000033,0.000059,0.000017,0.000045,0.000027,0.000017,0.000049,0.000041,0.000064,0.000044,0.000036,0.000028,0.000024,0.000031,0.000068,0.000041,0.000050
3,3,1854245,0.000046,0.000029,0.000072,0.000045,0.295399,0.000029,0.049259,0.000042,0.000017,0.000076,0.000043,0.110848,0.000071,0.000036,0.000065,0.000042,0.000007,0.000014,0.000033,0.000022,0.000022,0.000037,0.000063,0.000044,0.000056,0.000050,0.000051,0.000021,0.000023,0.000012,0.000014,0.000051,0.000035,0.000054,0.000043,0.000041,0.000078,0.0

**In above dataframe, "index" column is the row number, "id" column is the sample/document/text id, the same with "id" in dataset, "0" to max topic number columns are the weight of each topic per document.**

### 1.2 Add Dominant Topics Column

**Find dominant topic of each document:**

In [8]:
# https://stackoverflow.com/questions/46819405/how-to-get-the-name-of-column-with-maximum-value-in-pyspark-dataframe

def argmax(cols, *args):
    return [c for c, v in zip(cols, args) if v == max(args)][0]

def add_domtopic(df):
    """
    find the dominant topic of each sample/row/document
    input: dataframe of weight of each topic
    output: the dominant topic number dataframe
    """
    argmax_udf = lambda cols: F.udf(lambda *args: argmax(cols, *args), StringType())
    return (df
            .withColumn('domtopic',
                        argmax_udf(df.columns[2:])(*df.columns[2:]))
            .withColumn('weight', 
                        F.greatest(*[F.col(x) for x in df.columns[2:-1]])))

# add the df_dominant to doc-topic matrix
df_doctopic = add_domtopic(df_doctopic)

In [9]:
#print('Shape of dataframe: ({}, {})'.format(df_doctopic.count(), len(df_doctopic.columns)))

#df_doctopic.limit(5).toPandas().head()

### 1.3 Add Metadata Columns

**Here we only add "region" and "date" column as metadata, the accuracy of time could achieve "day" level, using the time series features we could convert "date" to year base or month base etc. depending on the need.**

In [10]:
df_doctopic = (df_doctopic
               .join(df_meta, df_doctopic.id == df_meta.id_)
               .withColumn('year', F.date_format('date', 'yyyy'))
               .drop('id_')
               .drop('date')
               .orderBy('index'))

In [11]:
#print('Shape of dataframe: ({}, {})'.format(df_doctopic.count(), len(df_doctopic.columns)))

#df_doctopic.limit(5).toPandas().head()

### 1.4 Document - Dominant Topics Dataframe

**It is hard to plot the doc-topic matrix (high dimension), we need transform it to extract or reduce features. First we generate dominant topics dataframe which could be used to reveal the relationship between dominant topics and region/year.**

In [12]:
df_docdomtopic = (df_doctopic
                  .join(df_topics, df_doctopic.domtopic == df_topics.topic)
                  .select(F.col('id'), 
                          F.col('region'), 
                          F.col('year'), 
                          F.col('domtopic'), 
                          F.col('weight'), 
                          F.col('words'))
                  .orderBy('id'))

df_docdomtopic.cache()

DataFrame[id: int, region: string, year: string, domtopic: string, weight: double, words: string]

In [13]:
print('Shape of dataframe: ({}, {})'.format(df_docdomtopic.count(), len(df_docdomtopic.columns)))

df_docdomtopic.limit(5).toPandas().head()

Shape of dataframe: (1511227, 6)


,id,region,year,domtopic,weight,words
0,1854215,Auckland,1862,100,0.290705,advertisements office news times paper published column advertising subscribers notice
1,1854224,Auckland,1862,35,0.966307,love life heart day thy world eyes long man night
2,1854233,Auckland,1863,146,0.382336,business public attention begs orders stock goods notice premises inform
3,1854245,Auckland,1863,4,0.295399,freight apply passage steam company passengers new_zealand tons agents wellington
4,1854248,Auckland,1863,123,0.569472,public men people man good time law fact great case


**Save the dataframe for later use:**

In [14]:
path = r'../models/train/domTopics'

df_docdomtopic.write.csv(path, mode='overwrite')

df_docdomtopic.unpersist()

DataFrame[id: int, region: string, year: string, domtopic: string, weight: double, words: string]

In [15]:
%%bash -s "$path"

cat $1/*.csv > $1/domTopics.csv

mv $1/domTopics.csv $1/../

rm -rf $1

### 1.5 Average Weight Topics Dataframe

**Beside dominant topics dataframe, we could calculate average weight of each topic in a year and create dataframe for it, which could reveal the weight variety of each topic as time goes on, we could take the average weight as features and execute data mining algorithms to find patterns, e.g. correlation between features. The weights of each topic were already scaled to 0-1 by default, so the sum of average weight of each year is 1, we do not need to scale it.**

In [16]:
df_avgweight = (df_doctopic
                .drop('index')
                .drop('id')
                .drop('domtopic')
                .drop('region')
                .drop('weight')
                .groupBy('year')
                .avg()
                .orderBy('year'))

In [17]:
df_avgweight.limit(5).toPandas().head()

,year,avg(0),avg(1),avg(2),avg(3),avg(4),avg(5),avg(6),avg(7),avg(8),avg(9),avg(10),avg(11),avg(12),avg(13),avg(14),avg(15),avg(16),avg(17),avg(18),avg(19),avg(20),avg(21),avg(22),avg(23),avg(24),avg(25),avg(26),avg(27),avg(28),avg(29),avg(30),avg(31),avg(32),avg(33),avg(34),avg(35),avg(36),avg(37),avg(38),avg(39),avg(40),avg(41),avg(42),avg(43),avg(44),avg(45),avg(46),avg(47),avg(48),avg(49),avg(50),avg(51),avg(52),avg(53),avg(54),avg(55),avg(56),avg(57),avg(58),...,avg(140),avg(141),avg(142),avg(143),avg(144),avg(145),avg(146),avg(147),avg(148),avg(149),avg(150),avg(151),avg(152),avg(153),avg(154),avg(155),avg(156),avg(157),avg(158),avg(159),avg(160),avg(161),avg(162),avg(163),avg(164),avg(165),avg(166),avg(167),avg(168),avg(169),avg(170),avg(171),avg(172),avg(173),avg(174),avg(175),avg(176),avg(177),avg(178),avg(179),avg(180),avg(181),avg(182),avg(183),avg(184),avg(185),avg(186),avg(187),avg(188),avg(189),avg(190),avg(191),avg(192),avg(193),avg(194),avg(195),avg(196),avg(197),avg(198),avg(199)
0,1839,0.000022,0.000014,0.000034,0.000021,0.000012,0.000014,0.057935,0.000020,0.000008,0.000036,0.000020,0.000039,0.000033,0.000017,0.000031,0.000020,0.000003,0.000007,0.000016,0.000010,0.000010,0.000018,0.000030,0.000021,0.000026,0.000024,0.022170,0.000010,0.000011,0.000005,0.000006,0.000024,0.000017,0.000026,0.000020,0.000019,0.000037,0.000020,0.025340,0.000042,0.000018,0.000024,0.000025,0.000021,0.000014,0.000011,0.056271,0.000022,0.000009,0.000020,0.000032,0.000006,0.000015,0.000011,0.000019,0.000019,0.000021,0.000033,0.000025,...,0.000019,0.000016,0.000042,0.071576,0.000019,0.000029,0.042758,0.000020,0.000018,0.000029,0.000033,0.000024,0.000025,0.000047,0.200251,0.000019,0.000021,0.000019,0.000027,0.000015,0.000019,0.126434,0.000021,0.000026,0.000013,0.000025,0.000016,0.000013,0.059638,0.000019,0.000008,0.058656,0.000035,0.000009,0.000029,0.000014,0.000048,0.000032,0.000038,0.000018,0.000014,0.000019,0.000018,0.000014,0.000025,0.000007,0.000019,0.000012,0.000007,0.000021,0.000017,0.000027,0.000019,0.013644,0.000012,0.054292,0.000013,0.042617,0.000018,0.000021
1,1840,0.004957,0.000078,0.003129,0.000122,0.010571,0.000287,0.000082,0.000537,0.051494,0.000208,0.000117,0.022553,0.009196,0.007969,0.000177,0.000167,0.000020,0.000208,0.011346,0.005784,0.000060,0.002791,0.002794,0.000365,0.002917,0.002375,0.003618,0.002110,0.000104,0.006520,0.000037,0.003566,0.005433,0.000791,0.000117,0.000112,0.000214,0.000314,0.002645,0.002665,0.007444,0.000500,0.001434,0.000122,0.001569,0.000063,0.032458,0.000129,0.000495,0.005863,0.000657,0.000032,0.003670,0.001765,0.000110,0.001529,0.000488,0.008640,0.000407,...,0.000107,0.000091,0.004596,0.002989,0.000390,0.002164,0.017940,0.000192,0.000289,0.000692,0.000464,0.004884,0.002994,0.000913,0.030039,0.000320,0.007348,0.000111,0.000154,0.003058,0.007865,0.095496,0.001101,0.004692,0.000161,0.000412,0.000143,0.000497,0.012058,0.000109,0.000046,0.008002,0.000285,0.000050,0.007430,0.000078,0.012094,0.020500,0.000241,0.000102,0.000122,0.000460,0.013739,0.000082,0.008174,0.000042,0.000111,0.000109,0.000041,0.000120,0.001090,0.000661,0.001183,0.091319,0.002719,0.000058,0.004373,0.007108,0.002331,0.000123
2,1841,0.000080,0.000050,0.010016,0.000970,0.011617,0.001214,0.000052,0.000093,0.090255,0.000407,0.000074,0.023536,0.000434,0.021879,0.004041,0.001360,0.000013,0.001702,0.006477,0.008877,0.000214,0.003883,0.002949,0.000076,0.008997,0.000087,0.002273,0.000036,0.000103,0.000078,0.000023,0.002260,0.004891,0.000473,0.000104,0.001287,0.000176,0.005409,0.000366,0.000828,0.004210,0.002620,0.000127,0.000306,0.000052,0.001875,0.035065,0.002064,0.001628,0.003541,0.000364,0.001355,0.002565,0.000039,0.000108,0.000375,0.000123,0.000413,0.000334,...,0.000068,0.001551,0.000730,0.001200,0.000446,0.001001,0.039536,0.001619,0.000066,0.000107,0.005406,0.000118,0.019244,0.000443,0.010021,0.000111,0.020640,0.000155,0.000561,0.000055,0.003953,0.074946,0.009404,0.000094,0.000377,0.000336,0.000206,0.001508,0.076639,0.000728,0.000029,0.0

**check years are identical with dataset:**

In [18]:
print('Year range of average weight dataframe:', 
      df_avgweight.select('year').collect())
print('Year range of doc-topic      dataframe:', 
      df_doctopic.select('year').collect())

Year range of average weight dataframe: Row(min(year)='1839', max(year)='1945') 
Year number: 104
Year range of doc-topic      dataframe: Row(min(year)='1839', max(year)='1945') 
Year number: 104


**The dimension of the avrage weight dataframe is small (topic_n\*year_n), so we directly save the dataframe using Pandas:**

In [19]:
path = r'../models/train/avgWeight.csv'

df_avgweight.toPandas().to_csv(path, header=False, index=False, encoding='utf-8')

## 2 Prepare Dataframe for Subset

**The process for subset is the same with for full dataset, we wrapped the process to a function and call it for each subset.**

### 2.1 By Range of Time

**Generate new weight for WWI.**

In [48]:
path = r'../models/wwi/docTopicsInfer.txt'

df_doctopic = load_doctopic(path, topic_number, spark)
df_weight = df_doctopic.groupBy().sum().toPandas()
df_weight.drop(df_weight.columns[[0, 1]], axis=1, inplace=True)
df_weight = df_weight.T.reset_index(drop=True)
df_weight.columns = ['weight']
df_weight = df_weight / df_weight.sum()
df_topics_wwi = df_topics.toPandas().join(df_weight).drop(columns='weight_')
df_topics_wwi = df_topics_wwi[['topic', 'weight', 'words']]

print('Shape of dataframe:', df_topics_wwi.shape)
#df_topics_wwi.head()

Shape of dataframe: (200, 3)


,topic,weight,words
0,0,0.002288,council city borough mayor town water street councillor works supply
1,1,0.001280,blenheim marlborough picton street wairau griffiths market road high havelock
2,2,0.051251,years man time london made ago great lady day young
3,3,0.003137,wool sheep stock company agents sale produce dip sales grain
4,4,0.001332,freight apply passage steam company passengers new_zealand tons agents wellington


In [50]:
path = r'../models/wwi/topicKeys.csv'
df_topics_wwi.to_csv(path, header=False, index=False, encoding='utf-8')

**Generate other dataframes:**

In [20]:
df_docdomtopic, df_avgweight = preplot(df_doctopic, df_meta, df_topics, 'month')

df_docdomtopic.cache()

**Save dataframes:**

In [21]:
path = r'../models/wwi/avgWeight.csv'
df_avgweight.toPandas().to_csv(path, header=False, index=False, encoding='utf-8')

path = r'../models/wwi/domTopics/'
df_docdomtopic.write.csv(path, mode='overwrite')

df_docdomtopic.unpersist()

**Convert multi files to a csv file:**

In [22]:
%%bash -s "$path"

cat $1/*.csv > $1/domTopics.csv

mv $1/domTopics.csv $1/../

rm -rf $1

### 2.2 By Region

**Generate new weight for regions.**

In [51]:
path = r'../models/regions/docTopicsInfer.txt'

df_doctopic = load_doctopic(path, topic_number, spark)
df_weight = df_doctopic.groupBy().sum().toPandas()
df_weight.drop(df_weight.columns[[0, 1]], axis=1, inplace=True)
df_weight = df_weight.T.reset_index(drop=True)
df_weight.columns = ['weight']
df_weight = df_weight / df_weight.sum()
df_topics_regions = df_topics.toPandas().join(df_weight).drop(columns='weight_')
df_topics_regions = df_topics_regions[['topic', 'weight', 'words']]

print('Shape of dataframe:', df_topics_regions.shape)
#df_topics_regions.head()

Shape of dataframe: (200, 3)


,topic,weight,words
0,0,0.004762,council city borough mayor town water street councillor works supply
1,1,0.000868,blenheim marlborough picton street wairau griffiths market road high havelock
2,2,0.050096,years man time london made ago great lady day young
3,3,0.002568,wool sheep stock company agents sale produce dip sales grain
4,4,0.003922,freight apply passage steam company passengers new_zealand tons agents wellington


In [52]:
path = r'../models/regions/topicKeys.csv'
df_topics_regions.to_csv(path, header=False, index=False, encoding='utf-8')

**Generate other dataframes:**

In [23]:
file = r'../models/regions/docTopicsInfer.txt'

df_docdomtopic, df_avgweight = preplot(df_doctopic, df_meta, df_topics, 'year')

df_docdomtopic.cache()

**Save dataframes:**

In [24]:
path = r'../models/regions/avgWeight.csv'
df_avgweight.toPandas().to_csv(path, header=False, index=False, encoding='utf-8')

path = r'../models/regions/domTopics/'
df_docdomtopic.write.csv(path, mode='overwrite')

df_docdomtopic.unpersist()

**Convert multi files to a csv file:**

In [25]:
%%bash -s "$path"

cat $1/*.csv > $1/domTopics.csv

mv $1/domTopics.csv $1/../

rm -rf $1

### 2.3 By Label

**Generate new weight for ADs.**

In [54]:
path = r'../models/ads/docTopicsInfer.txt'

df_doctopic = load_doctopic(path, topic_number, spark)
df_weight = df_doctopic.groupBy().sum().toPandas()
df_weight.drop(df_weight.columns[[0, 1]], axis=1, inplace=True)
df_weight = df_weight.T.reset_index(drop=True)
df_weight.columns = ['weight']
df_weight = df_weight / df_weight.sum()
df_topics_ads = df_topics.toPandas().join(df_weight).drop(columns='weight_')
df_topics_ads = df_topics_ads[['topic', 'weight', 'words']]

print('Shape of dataframe:', df_topics_ads.shape)
#df_topics_ads.head()

Shape of dataframe: (200, 3)


,topic,weight,words
0,0,0.002107,council city borough mayor town water street councillor works supply
1,1,0.001009,blenheim marlborough picton street wairau griffiths market road high havelock
2,2,0.055509,years man time london made ago great lady day young
3,3,0.002214,wool sheep stock company agents sale produce dip sales grain
4,4,0.005585,freight apply passage steam company passengers new_zealand tons agents wellington


In [55]:
path = r'../models/ads/topicKeys.csv'
df_topics_ads.to_csv(path, header=False, index=False, encoding='utf-8')

**Generate other dataframes:**

In [26]:
file = r'../models/ads/docTopicsInfer.txt'

df_docdomtopic, df_avgweight = preplot(df_doctopic, df_meta, df_topics, 'year')

df_docdomtopic.cache()

**Save dataframes:**

In [27]:
path = r'../models/ads/avgWeight.csv'
df_avgweight.toPandas().to_csv(path, header=False, index=False, encoding='utf-8')

path = r'../models/ads/domTopics'
df_docdomtopic.write.csv(path, mode='overwrite')

df_docdomtopic.unpersist()

**Convert multi files to a csv file:**

In [28]:
%%bash -s "$path"

cat $1/*.csv > $1/domTopics.csv

mv $1/domTopics.csv $1/../

rm -rf $1

---